In [57]:
import pandas as pd
import re
from io import StringIO
import numpy as np
import base64

In [58]:
# def create_onedrive_directdownload (onedrive_link):
#     data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
#     data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
#     resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
#     return resultUrl

# onedrive_link=create_onedrive_directdownload("https://1drv.ms/x/s!AoiE7xOoBAsngsd-PpntYUKRt9nOQg?e=YO5X5h")
# print(onedrive_link)

# df=pd.read_excel(onedrive_link)
# df

df = pd.read_csv('labelled_enron.csv')
df

,column_a,message_id,date,from,to,subject,x_from,x_to,x_cc,x_bcc,...,cat_10_level_1,cat_10_level_2,cat_10_weight,cat_11_level_1,cat_11_level_2,cat_11_weight,cat_12_level_1,cat_12_level_2,cat_12_weight,labeled
0,379,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15T14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,381,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15T14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2139,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20T17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2140,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20T17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2232,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09T12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,499650,<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05T13:42:45,frozenset({'louise.kitchen@enron.com'}),"frozenset({'greg.whalley@enron.com', 'john.lav...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1698,501042,<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13T14:00:00,frozenset({'steven.kean@enron.com'}),"frozenset({'greg.whalley@enron.com', 'kenneth....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1699,507376,<205897.1075861997314.JavaMail.evans@thyme>,2001-11-13T22:20:01,frozenset({'jeanie.slone@enron.com'}),frozenset({'mark.whitt@enron.com'}),RE: confidential ee info,"Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1700,509660,<23743848.1075863311776.JavaMail.evans@thyme>,2001-07-11T15:29:22,frozenset({'legalonline-compliance@enron.com'}),"frozenset({'williams@mailman.enron.com', 'bwil...",Confidential Information and Securities Trading,Office of the Chairman - Enron Wholesale Servi...,"WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [59]:
df=df[~df['content'].isna()]
df=df[df['labeled']==True]
col=['message_id','subject','content','cat_1_level_2']
df=df.loc[:,col]
df.head()

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,1.0
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,3.0
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",4.0
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",4.0
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",4.0


In [60]:
# Create a new 'label' column based on 'cat_1_level_2'
df['label'] = df['cat_1_level_2'].apply(lambda x: 1.0 + (x / 10.0))

# Map 'label' values to the desired labels
label_mapping = {
    1.1: 'Company Business, Strategy, etc.(elaborate in Section 3 [Topics])',
    1.2: 'Purely Personal',
    1.3: 'Personal but in professional context(e.g., it was good working with you)',
    1.4: 'Logistic Arrangements(meeting scheduling, technical support, etc)',
    1.5: 'Employment arrangements(job seeking, hiring, recommendations, etc) ',
    1.6: 'Document editing/checking(collaboration)',
    1.7: 'Empty message(due to missing attachment)',
    1.8: 'Empty message'
}

df['label'] = df['label'].map(label_mapping)
df.drop(columns=['cat_1_level_2'], inplace=True)
df

,message_id,subject,content,label
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,"Company Business, Strategy, etc.(elaborate in ..."
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,"Personal but in professional context(e.g., it ..."
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...","Logistic Arrangements(meeting scheduling, tech..."
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...","Logistic Arrangements(meeting scheduling, tech..."
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...","Logistic Arrangements(meeting scheduling, tech..."
...,...,...,...,...
1697,<18361957.1075861368310.JavaMail.evans@thyme>,Confidential Due Diligence Info - GS,There is a lot of confidential information her...,"Company Business, Strategy, etc.(elaborate in ..."
1698,<29049822.1075842045435.JavaMail.evans@thyme>,California's dim bulbs,You may have seen this when it first came out ...,"Company Business, Strategy, etc.(elaborate in ..."
1699,<205897.1075861997314.JavaMail.evans@thyme>,RE: confidential ee info,"Mark, Can you email me or fax me any written d...","Employment arrangements(job seeking, hiring, r..."
1700,<23743848.1075863311776.JavaMail.evans@thyme>,Confidential Information and Securities Trading,"To:WILLIAMS, WILLIAM Email:bwillia5@enron.com ...","Company Business, Strategy, etc.(elaborate in ..."


In [61]:
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)
df['content']=df['content'].replace({',':'',},regex=True)
df['content']=df['content'].replace({';':'',},regex=True)

df['content']=df['content'].replace({'"':'',},regex=True)
df['content']=df['content'].replace({'"':''},regex=True)

df['subject']=df['subject'].replace({"'":""},regex=True)
df['subject']=df['subject'].replace({"'":""},regex=True)

df=df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[df['content'].str.contains('-----Original Message-----'),'content']=df.loc[df['content'].str.contains('-----Original Message-----'),'content'].str.extract(r'([\s\S]*?)'+r'-----Original Message-----',expand=False)
df.loc[df['content'].str.contains('----- Forwarded by'),'content']=df.loc[df['content'].str.contains('----- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'----- Forwarded by',expand=False)
df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content']=df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'---------------------- Forwarded by',expand=False)
df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content']=df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content'].str.extract(r'([\s\S]*?)'+r'-------------------------- Sent from my BlackBerry',expand=False)

df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)
df.to_csv('simplified_labeled_enron.csv', index=False)
df.head()

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_26864/2581723703.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)


,message_id,subject,body,category
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,"Company Business, Strategy, etc.(elaborate in ..."
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,"Personal but in professional context(e.g., it ..."
5,<21261996.1075858638025.JavaMail.evans@thyme>,Re: Western Wholesale Activities - Gas & Power...,Can you guys coordinate to make sure someone l...,"Logistic Arrangements(meeting scheduling, tech..."
6,<20399547.1075857614321.JavaMail.evans@thyme>,Re: confidential employee information-dutch qu...,thx Jeanie Slone 12/19/2000 04:51 PM To: John ...,"Employment arrangements(job seeking, hiring, r..."
7,<860767.1075849626951.JavaMail.evans@thyme>,Re: HP -- confidential internal document,This is an excellent update. Thanks for puttin...,"Company Business, Strategy, etc.(elaborate in ..."


In [62]:
# Split into train and test dataset
original_df = pd.read_csv('simplified_labeled_enron.csv')

# Shuffle the rows randomly
shuffled_df = original_df.sample(frac=1, random_state=42)  # Use a fixed random_state for reproducibility

# Split the DataFrame into two parts
train_df = shuffled_df.iloc[:1400]
test_df = shuffled_df.iloc[1400:]

# Save the two split DataFrames to separate CSV files
train_df.to_csv('simplified_labeled_enron_train.csv', index=False)
test_df.to_csv('simplified_labeled_enron_test.csv', index=False)

In [63]:
# creating prompts for fine-tuning the model
train_df['prompts'] = "<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: " + train_df['body'] + " [/INST] Category: " + train_df['category']
train_df['prompts'][2]

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_26864/2759820681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['prompts'] = "<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: " + train_df['body'] + " [/INST] Category: " + trai

"<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: Can you guys coordinate to make sure someone listens to this conference call each week. Tara from the fundamental group was recording these calls when they happened every day. Phillip ----------------- [/INST] Category: Logistic Arrangements(meeting scheduling, technical support, etc)"

In [64]:
# creating training dataset

prompts_df = pd.DataFrame(columns=['prompts'])
prompts_df['prompts'] = train_df['prompts'].astype(str)
prompts_df = prompts_df.drop(prompts_df[prompts_df['prompts'] == 'nan'].index)
prompts_df.reset_index()
prompts_df.to_csv('prompts_train.csv', index=False)
prompts_df

,prompts
1524,<s>[INST] <<SYS>> I am sharing an email body w...
1483,<s>[INST] <<SYS>> I am sharing an email body w...
668,<s>[INST] <<SYS>> I am sharing an email body w...
1422,<s>[INST] <<SYS>> I am sharing an email body w...
244,<s>[INST] <<SYS>> I am sharing an email body w...
...,...
1109,<s>[INST] <<SYS>> I am sharing an email body w...
1266,<s>[INST] <<SYS>> I am sharing an email body w...
1183,<s>[INST] <<SYS>> I am sharing an email body w...
524,<s>[INST] <<SYS>> I am sharing an email body w...


In [65]:
prompts_df[prompts_df['prompts']=='nan']

,prompts


In [66]:
# # creating testing dataset

test_df['input_test_prompts'] = "<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: " + test_df['body'] + " [/INST] Category: "
prompts_df_test = pd.DataFrame(columns=['input_test_prompts', 'category'])
prompts_df_test['input_test_prompts'] = test_df['input_test_prompts'].astype(str)
prompts_df_test['category'] = test_df['category'].astype(str)
prompts_df_test = prompts_df_test.drop(prompts_df_test[prompts_df_test['input_test_prompts'] == 'nan'].index)
prompts_df_test.reset_index()
prompts_df_test.to_csv('prompts_test.csv', index=False)
prompts_df_test

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_26864/295945295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['input_test_prompts'] = "<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: " + test_df['body'] + " [/INST] Category: 

,input_test_prompts,category
540,<s>[INST] <<SYS>> I am sharing an email body w...,"Personal but in professional context(e.g., it ..."
1194,<s>[INST] <<SYS>> I am sharing an email body w...,"Logistic Arrangements(meeting scheduling, tech..."
748,<s>[INST] <<SYS>> I am sharing an email body w...,"Employment arrangements(job seeking, hiring, r..."
1440,<s>[INST] <<SYS>> I am sharing an email body w...,"Company Business, Strategy, etc.(elaborate in ..."
484,<s>[INST] <<SYS>> I am sharing an email body w...,"Company Business, Strategy, etc.(elaborate in ..."
...,...,...
1130,<s>[INST] <<SYS>> I am sharing an email body w...,Document editing/checking(collaboration)
1294,<s>[INST] <<SYS>> I am sharing an email body w...,"Company Business, Strategy, etc.(elaborate in ..."
860,<s>[INST] <<SYS>> I am sharing an email body w...,"Company Business, Strategy, etc.(elaborate in ..."
1459,<s>[INST] <<SYS>> I am sharing an email body w...,"Company Business, Strategy, etc.(elaborate in ..."


Checking a few prompts...

In [67]:
prompts_df_test['input_test_prompts'][1459]

"<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: The EPSA Leg. Affairs Committee is having a special meeting this afternoon (out of the normal biweekly cycle) to discuss how EPSA should respond to the Barton RTO discussion draft. In addition to the e-mail yesterday morning I spoke in person with Gene Peters of the EPSA staff to convey our strongly held view that mere technical corrections were not enough to fix the Barton RTO proposal as the EPSA draft letter stated. Gene came around becau

In [68]:
prompts_df_test['input_test_prompts'][1126]

"<s>[INST] <<SYS>> I am sharing an email body with you. Based on the text in the body, you need to classify the email in one of the following eight categories: 'Company Business, Strategy, etc.'; 'Purely Personal'; 'Personal but in professional context (e.g., it was good working with you)'; 'Logistic Arrangements (meeting scheduling, technical support, etc)'; 'Employment arrangements (job seeking, hiring, recommendations, etc)'; 'Document editing/checking (collaboration)'; 'Empty message (due to missing attachment)'; 'Empty message'. <</SYS>> Mail: Thanks for getting on top of this. Please get the word out to the HR team leaders as soon as you know more. From: Aaron Brown/ENRON@enronXgate on 07/13/2001 11:36 AM To: Steven J Kean/NA/Enron@Enron Mary Joyce/ENRON@enronXgate cc: Subject: Enron's PaineWebber Service Rocky Emery the Dowd family Kim Bowden and another 15 or so employees of UBSPaineWebber (our stock tracking provider...options restricted stock etc) left yesterday evening witho